In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)
from conf import *

In [2]:
length = [12, 12]
hex_ = nk.machine.new_hex(np.array(length))

kernel = 1
# sweep_size = 200
decay_factor = 1  # or 'sigmoid decay'



g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)



op_transition1 = f.dimer_flip1(length = np.array(length))


In [3]:
ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

2


In [5]:
V = 1.0
h = 0.9

ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))


n_jobs = 12
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition1
    , length = length, n_chains=n_chains, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sampler = sa_mul.sa_list[0]

yoyo
number of core : 12
accepted ratio :  0.8948019801980198
accepted ratio :  0.913151364764268
accepted ratio :  0.905940594059406
accepted ratio :  0.9143920595533499
accepted ratio :  0.8825
accepted ratio :  0.90875
accepted ratio :  0.9121287128712872
accepted ratio :  0.9
accepted ratio :  0.9265255292652553
accepted ratio :  0.904228855721393


In [18]:
%time P_pirime = op_transition1.get_conn(P[0])[0]

xs_n 1  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 1  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 8  n_conn 0
xs_n 3  n_conn 0
xs_n 12  n_conn 0
xs_n 7  n_conn 0
xs_n 7  n_conn 0
xs_n 15  n_conn 0
xs_n 14  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 7  n_conn 0
xs_n 12  n_conn 0
xs_n 3  n_conn 0
xs_n 8  n_conn 0
xs_n 8  n_conn 0
xs_n 0  n_conn 0
xs_n 1  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 8  n_conn 0
xs_n 3  n_conn 0
xs_n 4  n_conn 0
xs_n 4  n_conn 0
xs_n 3  n_conn 0
xs_n 7  n_conn 0
xs_n 6  n_conn 0
xs_n 12  n_conn 0
xs_n 3  n_conn 0
xs_n 8  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 1  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 8  n_conn 0
xs_n 3  n_conn 0
xs_n 4  n_conn 0
xs_n 4  n_conn 0
xs_n 3  n_conn 0
xs_n 7  n_conn 0
xs_n 6  n_conn 0
xs_n

In [24]:
%time ma.log_val(P[0].reshape(1,-1))

CPU times: user 2.05 ms, sys: 705 µs, total: 2.75 ms
Wall time: 221 µs


array([7.78092483+0.j])

In [9]:
P = P[:,0,:]

In [12]:
(P[1] - P[0]).shape

(288,)

In [80]:
ma.log_val(P[:,:])

array([3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
       3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j, 3.87753514+0.j,
      

In [44]:
V = 1.0
h = 0.9

ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))


n_jobs = 12
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition1
    , length = length, n_chains=n_chains, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)

sr = nk.optimizer.SR(ma, diag_shift=5e-3)
opt = nk.optimizer.Sgd(ma, learning_rate=0.05, decay_factor = decay_factor ,N = n_iter)

gs = nk.Vmc(
hamiltonian=ham,
sampler=sa_mul,
optimizer=opt,
n_samples=10**3,
sr=sr,
n_discard=12*12,
)


gs.run(n_iter=n_iter)
# ma.save(parentdir + '/save/ma/'+name)

yoyo
number of core : 12


  0%|          | 0/1000 [00:00<?, ?it/s]

discard =  144
No output specified (out=[apath|nk.logging.JsonLogger(...)]).Running the optimization but not saving the output.
n_discard =  144
discard samples 104.932781457901
n_samples_node =  1000
generate_samples :  55.569984912872314
     prepare getting local values 0.09188270568847656
     obtained local values 0.17357778549194336
get_mc_stats 0.20006656646728516


  0%|          | 1/1000 [02:41<44:41:53, 161.07s/it, Energy=4.164+0.000j ± 0.020 [σ²=0.478]]

--->> cal O and jac  0.2505757808685303
compute_update  0.37010741233825684
(0.0052273923952437075+0j)
n_discard =  144


  0%|          | 1/1000 [02:59<49:40:37, 179.02s/it, Energy=4.164+0.000j ± 0.020 [σ²=0.478]]

Received keyboardinterrupt



KeyboardInterrupt: 

Received keyboardinterrupt

Received keyboardinterrupt
Received keyboardinterrupt

Received keyboardinterrupt
Received keyboardinterrupt
Received keyboardinterrupt


Received keyboardinterrupt

Received keyboardinterrupt



Received keyboardinterrupt

Received keyboardinterrupt

Received keyboardinterrupt



In [16]:
12*12*5

720